In [1]:
import re
import os
if not 'id_0123456789876543210' in locals():
    _rootlevel = 1
    _oldwd = re.sub(r'\\', '/', os.getcwd())
    _spdirs = _oldwd.split('/')
    _newwd = '/'.join(_spdirs[:(len(_spdirs)-_rootlevel)])
    os.chdir(_newwd)
    id_0123456789876543210 = None

In [2]:
import pickle
import numpy as np


def load_object(file_path):
    with open(file_path, "rb") as file:
        loaded_object = pickle.load(file)
    return loaded_object

In [3]:
from src.python.training.validation import Foldify, eval_model
from src.python.model import classification_model, validate_classification_model_hparams
from src.python.minimize.binary import floatBinary, intBinary, catBinary
from src.python.minimize.algorithms import SearchSpace, Function, random_search

In [8]:
class HparamSearchSpace(SearchSpace):
    def __init__(self, binary, seed=None):
        super().__init__(binary, seed)
        binary_dict = {x.name: x for x in self.binary}
        neurons = binary_dict['neurons']
        deep = binary_dict['deep']
        nofactible = [d > int(1 + np.log(n / 2.0) / np.log(2))
                      for n in range(neurons.a, neurons.b + 1)
                      for d in range(deep.a, deep.b + 1)]
        self.binary.dimension -= (neurons.dimension + deep.dimension) * \
            (sum(nofactible) / len(nofactible))

    def is_feasible(self, ndbits):
        hparam = ndbits.value_dict
        maxdeep = int(1 + np.log(hparam['neurons'] / 2.0) / np.log(2))
        return hparam['deep'] <= maxdeep


search_space = HparamSearchSpace(
    [
        intBinary(10, 10, name='input_dim'),
        intBinary(1, 2**8, digits=0, name='neurons'),
        intBinary(1, 8, digits=0, name='deep'),
        catBinary({'Adam', 'RMSprop', 'Nadam', 'SGD'}, name='optimizer'),
        floatBinary(0.00001, 0.1, digits=5, name='learning_rate'),
        catBinary({'relu', 'elu', 'LeakyReLU'}, name='activation'),
        catBinary({'uniform', 'normal'}, name='initializer'),
        intBinary(1, 2**7, digits=0, name='batch_size'),
        intBinary(25, 30, digits=0, name='epochs'),
        catBinary({True, False}, name='batch_normalization'),
        catBinary({'None'}, name='regularization'),
        catBinary({False}, name='dropout')
    ])


data = load_object('data/covid/classification/datasets/graves_train-1.1.pkl')

foldify = Foldify(10,  # len(data['x']),
                  nfolds=1,  # 6,
                  val_prop=0.2,
                  ftype='random',
                  sorted=True,
                  seed=555,
                  weight=data['sample'],
                  label=data['label'],
                  datasets=(data['x'], data['y']))


def fun(**hparam):
    validate_classification_model_hparams(hparam)
    eval = eval_model(model=classification_model,
                      hparam=hparam,
                      foldify=foldify,
                      infile='data/covid/classification/hpsearch/hpsearch_memory_graves-1.1.pkl',
                      outfile=None,
                      origin='dell inspiron')
    return eval['history'][-1]['val_mae'].iloc[-1]


random_search(Function(fun, argstype='kwargs'),
              search_space,
              seed=123,
              itermax=1,
              initialize=False,######
              infile='data/covid/classification/hpsearch/random_search_graves-1.1.0.pkl',
              outfile=None,
              verbose=3)

Hyperparameters:
	 input_dim: 10   neurons: 214   deep: 2   optimizer: RMSprop   learning_rate: 0.040486938289690536   activation: elu   initializer: uniform   batch_size: 20   epochs: 26   batch_normalization: True   regularization: None   dropout: False   regularization_factor: 0.0   dropout_rate: 0.0
Fold 1/1 ...
	 time: 0h 0m 3s   loss: 0.036291   mae: 0.161758   val_loss: 0.105579   val_mae: 0.294186
Total:
	 time: 0h 0m 3s   loss: 0.036291   mae: 0.161758   val_loss: 0.105579   val_mae: 0.294186

Progress =  {
    "iter": 1,
    "eval": 1,
    "time": 3.4428961277008057,
    "fmin": 0.29418641328811646,
    "xmin": {
        "input_dim": 10,
        "neurons": 214,
        "deep": 2,
        "optimizer": "RMSprop",
        "learning_rate": 0.040486938289690536,
        "activation": "elu",
        "initializer": "uniform",
        "batch_size": 20,
        "epochs": 26,
        "batch_normalization": true,
        "regularization": "None",
        "dropout": false
    }
} 



iter                                                       1
eval                                                       1
time                                                3.442896
fmin                                                0.294186
xmin       [, 11010101, 001, 0010, 01100111101000, 010, 0...
history       iter  eval      time      fmin         f  \...
dtype: object

------------------